In [1]:
# =============================================================================
# WORKSHOP: INTELIGENCIA GEOESPACIAL Y TEORÍA DE REDES EN COLOMBIA
# Docente: Jorge Iván Padilla-Buriticá | EAFIT 2026-1
# =============================================================================

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import networkx as nx
from scipy.spatial import distance_matrix

# 1. CARGA DE DATOS
# -----------------------------------------------------------------------------
df = pd.read_csv('colombia_geo_clean.csv')

# 2. VISUALIZACIÓN DINÁMICA SOBRE MAPA DE COLOMBIA
# -----------------------------------------------------------------------------
# Usaremos Plotly para un mapa interactivo.
# El color representa la temperatura y el tamaño el nivel de CO2.

fig_map = px.scatter_mapbox(
    df,
    lat="Latitud",
    lon="Longitud",
    color="Temperatura_C",
    size="Nivel_CO2",
    hover_name="Nombre_Punto",
    hover_data=["Departamento", "Region", "Elevacion_msnm"],
    color_continuous_scale=px.colors.cyclical.IceFire,
    zoom=4,
    height=700,
    title="Red de Monitoreo Ambiental Colombia - Mapa de Calor e Intensidad"
)

# Configuración del estilo del mapa (OpenStreetMap no requiere Token)
fig_map.update_layout(mapbox_style="open-street-map")
fig_map.update_layout(margin={"r":0,"t":50,"l":0,"b":0})
fig_map.show()

# 3. EJERCICIO DE GRAFOS: RED DE CONECTIVIDAD REGIONAL
# -----------------------------------------------------------------------------
# Tarea: Conectar nodos que estén a menos de 50km para simular una red mesh.

def generar_grafo_proximidad(df_region, radio_km=0.5):
    G = nx.Graph()

    # Añadir nodos con sus atributos
    for idx, row in df_region.iterrows():
        G.add_node(row['ID'], pos=(row['Longitud'], row['Latitud']), region=row['Region'])

    # Calcular distancias y crear aristas (edges) si están cerca
    coords = df_region[['Longitud', 'Latitud']].values
    dist_mat = distance_matrix(coords, coords)

    for i in range(len(coords)):
        for j in range(i + 1, len(coords)):
            if dist_mat[i, j] < radio_km:
                G.add_edge(df_region.iloc[i]['ID'], df_region.iloc[j]['ID'], weight=dist_mat[i,j])

    return G

# Filtramos por una región para no saturar el gráfico (Región Andina)
df_andina = df[df['Region'] == 'Andina'].head(60)
G_andina = generar_grafo_proximidad(df_andina)

# 4. TAREA DIDÁCTICA: CAMINOS MÍNIMOS Y CENTRALIDAD
# -----------------------------------------------------------------------------
# Vamos a encontrar la ruta más eficiente que conecte todos los puntos (MST)

mst = nx.minimum_spanning_tree(G_andina)
pos = nx.get_node_attributes(G_andina, 'pos')

# Visualización del Grafo sobre coordenadas reales
plt_fig = go.Figure()

# Dibujar Aristas del MST (Rutas Óptimas)
for edge in mst.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    plt_fig.add_trace(go.Scatter(
        x=[x0, x1], y=[y0, y1],
        mode='lines',
        line=dict(width=1, color='#888'),
        hoverinfo='none'
    ))

# Dibujar Nodos
plt_fig.add_trace(go.Scatter(
    x=[p[0] for p in pos.values()],
    y=[p[1] for p in pos.values()],
    mode='markers',
    marker=dict(size=10, color='orange', line_width=2),
    text=list(pos.keys())
))

plt_fig.update_layout(
    title="Topología de Red Óptima (MST) - Región Andina",
    showlegend=False,
    xaxis=dict(title="Longitud"),
    yaxis=dict(title="Latitud"),
    template="plotly_white"
)
plt_fig.show()

# 5. ANÁLISIS DE COMPLEJIDAD
# -----------------------------------------------------------------------------
print(f"Número de nodos activos: {G_andina.number_of_nodes()}")
print(f"Densidad de la red de comunicación: {nx.density(G_andina):.4f}")
print(f"Nodo con mayor importancia (Hub): {max(nx.degree_centrality(G_andina), key=nx.degree_centrality(G_andina).get)}")

Número de nodos activos: 60
Densidad de la red de comunicación: 0.0644
Nodo con mayor importancia (Hub): ST_217


In [4]:
# =============================================================================
# BLOQUE 6: REDES DE MAGNITUD EN LA COSTA CARIBE (VERSIÓN CORREGIDA)
# Propósito: Visualizar la intensidad de CO2 y la conectividad de sensores.
# =============================================================================

import plotly.graph_objects as go
import networkx as nx

# 1. Asegurar que los datos y la función estén presentes
df = pd.read_csv('colombia_geo_clean.csv')

# Definición de la función (repetida aquí para evitar errores de contexto)
def generar_grafo_proximidad(df_region, radio_km=0.7):
    G = nx.Graph()
    for idx, row in df_region.iterrows():
        G.add_node(row['ID'], pos=(row['Longitud'], row['Latitud']))
    coords = df_region[['Longitud', 'Latitud']].values
    dist_mat = distance_matrix(coords, coords)
    for i in range(len(coords)):
        for j in range(i + 1, len(coords)):
            if dist_mat[i, j] < radio_km:
                G.add_edge(df_region.iloc[i]['ID'], df_region.iloc[j]['ID'])
    return G

# 2. Filtrar datos para la Región Caribe
df_caribe = df[df['Region'] == 'Caribe'].copy()

# 3. Definir conexiones
G_caribe = generar_grafo_proximidad(df_caribe, radio_km=0.7)
pos_caribe = nx.get_node_attributes(G_caribe, 'pos')

# 4. Creación del Mapa con Plotly
fig_caribe = go.Figure()

# CAPA A: Aristas (Conexiones)
edge_x, edge_y = [], []
for edge in G_caribe.edges():
    # Validar que los nodos tengan posición antes de graficar
    if edge[0] in pos_caribe and edge[1] in pos_caribe:
        x0, y0 = pos_caribe[edge[0]]
        x1, y1 = pos_caribe[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

fig_caribe.add_trace(go.Scattermapbox(
    lat=edge_y, lon=edge_x,
    mode='lines',
    line=dict(width=1, color='rgba(150, 150, 150, 0.6)'),
    name="Vínculos de Comunicación",
    hoverinfo='none'
))

# CAPA B: Nodos (Sensores)
fig_caribe.add_trace(go.Scattermapbox(
    lat=df_caribe['Latitud'],
    lon=df_caribe['Longitud'],
    mode='markers',
    marker=go.scattermapbox.Marker(
        size=df_caribe['Demanda_Energia']/4, # Ajustado para visibilidad
        color=df_caribe['Nivel_CO2'],
        colorscale='YlOrRd',
        showscale=True,
        colorbar=dict(title="Nivel CO2 (ppm)", thickness=20)
    ),
    text=df_caribe['Nombre_Punto'],
    hoverinfo='text',
    name="Estaciones de Monitoreo"
))

# 5. Configuración de Layout (Ajustado para asegurar salida visual)
fig_caribe.update_layout(
    title="Análisis de Red y Magnitudes: Costa Caribe",
    mapbox=dict(
        style="open-street-map", # Cambio a estilo gratuito garantizado
        center=dict(lat=10.5, lon=-74.5),
        zoom=5.5
    ),
    margin={"r":0,"t":50,"l":0,"b":0},
    showlegend=True,
    template="plotly_dark"
)

# Mostrar la figura explícitamente
fig_caribe.show()

In [3]:
# =============================================================================
# BLOQUE 7: DETECCIÓN DE HUBS Y TOPOLOGÍA EN ANTIOQUIA
# Propósito: Identificar los nodos con mayor impacto en la red regional.
# =============================================================================

# 1. Filtrar datos para Antioquia
df_ant = df[df['Departamento'] == 'Antioquia'].copy()

# 2. Crear el grafo de infraestructura
G_ant = generar_grafo_proximidad(df_ant, radio_km=0.6)

# 3. Calcular Métrica de Centralidad (Importancia del Nodo)
centralidad = nx.degree_centrality(G_ant)
df_ant['Centralidad'] = df_ant['ID'].map(centralidad)

# 4. Visualización Geo-Gráfica
fig_ant = px.scatter_mapbox(
    df_ant,
    lat="Latitud", lon="Longitud",
    color="Centralidad",
    size="Centralidad",
    color_continuous_scale="Viridis",
    hover_name="Nombre_Punto",
    title="Topología de Red en Antioquia: Identificación de Nodos Críticos (Hubs)",
    zoom=7, height=600
)

# Añadir las aristas del grafo sobre el mapa
pos_ant = {row['ID']: (row['Longitud'], row['Latitud']) for _, row in df_ant.iterrows()}
for edge in G_ant.edges():
    x0, y0 = pos_ant[edge[0]]
    x1, y1 = pos_ant[edge[1]]
    fig_ant.add_trace(go.Scattermapbox(
        lat=[y0, y1], lon=[x0, x1],
        mode='lines',
        line=dict(width=2, color='white'),
        opacity=0.2,
        showlegend=False
    ))

fig_ant.update_layout(mapbox_style="carto-darkmatter")
fig_ant.show()

# =============================================================================
# RESUMEN PARA EL TALLER:
# 1. COLOR: Use escalas secuenciales para magnitudes (CO2) y cualitativas para regiones.
# 2. TAMAÑO: Reserve el tamaño para variables de impacto (Energía/Población).
# 3. CONEXIONES: Dibuje aristas con baja opacidad (0.2 - 0.5) para evitar el desorden.
# 4. INTERACTIVIDAD: El mapa debe permitir zoom para ver el 'micro-grafo' local.
# =============================================================================